# Raw and pre-calibrated data

**WARNING** work in progress

**Author:** Dr. Michele Peresano (CEA-Saclay/IRFU/DAp/LEPCHE), 2021

**Recommended datasample(s):** any simtel file (currently tested on Prod5b)

**Data level(s):** R0 and/or R1

**Description:**

This notebook provides benchmarks for R0 and R1 data levels, meaning waveforms and pre-calibrated, gain-selected waveforms respectively.
It also extracts distributions for quantities used in the pre-calibration process, e.g. digital counts to photoelectron factor and pedestals.

All benchmarks are shown per-gain-channel.

**Requirements and steps to reproduce:**

This notebook requires **ctapipe > 0.11.0 (at the time of writing this means the master branch)**.

`papermill benchmarks-TRAINING-calibration.ipynb results_benchmarks-TRAINING-calibration.ipynb`

adding `-p parameter value` to override any of the available parameters, which can be listed with `--help-notebook`.

**Development and testing:**  

As with any other part of _protopipe_ and being part of the official repository, this notebook can be further developed by any interested contributor.  
The execution of this notebook is not currently automatic, it must be done locally by the user _before_ pushing a pull-request.
Please, strip the output before pushing.

## Table of contents

- [DC to PHE](#DC-to-PHE)
- [Pedestals](#Pedestals)
- [Noise](#Noise)

## Imports
[back to top](#Table-of-contents)

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc
import matplotlib.style as style
from cycler import cycler

from ctapipe.instrument import SubarrayDescription
from ctapipe.io import EventSource

from protopipe.benchmarks.utils import add_stats, get_fig_size
from protopipe.benchmarks.operations import load_tel_id

## Input data
[back to top](#Table-of-contents)

In [ ]:
# simtel file
input_directory_simtel = Path("/Users/michele/Applications/ctasoft/dirac/shared_folder/productions/Prod5b/NSB1x/CTAN/az180/zd20")
input_file_name_simtel = "gamma_20deg_180deg_run100___cta-prod5b-lapalma_desert-2158m-LaPalma-dark.simtel.zst"
# HDF5 file produced with ctapipe-process
input_directory_h5 = "/Users/michele/Applications/ctasoft/dirac/shared_folder/analyses/v0.5.0a1_Prod5b_LaPalma_zd20_az180_NSB1x_point_source/data/DL1a/LocalPeakWindowSum"
input_file_name_h5 = "1stPass.dl1.h5"
# other
max_events = 1 # number of showers to use
allowed_telescopes = None # number of telescopes to condider (default: all telescopes)
use_seaborn = True # see benchmarks.yaml
plots_scale = None
output_directory = Path.cwd() # default: current working directory

In [ ]:
[use_seaborn] = string_to_boolean([use_seaborn])

In [ ]:
if input_directory_simtel:
    input_directory_simtel = Path(input_directory_simtel)
input_file_simtel = input_directory_simtel / input_file_name_simtel

if input_directory_h5:
    input_directory_h5 = Path(input_directory_h5)
input_file_h5 = input_directory_h5 / input_file_name_h5

In [ ]:
# First we check if a _plots_ folder exists already.  
# If not, we create it.
plots_folder = Path(output_directory) / "plots"
plots_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# Plot aesthetics settings

scale = matplotlib_settings["scale"] if plots_scale is None else float(plots_scale)

style.use(matplotlib_settings["style"])
cmap = matplotlib_settings["cmap"]
rc('font', size=matplotlib_settings["rc"]["font_size"])

if matplotlib_settings["style"] == "seaborn-colorblind":
    
    # Change color order to have first ones more readable
    colors_order = ['#0072B2', '#D55E00', '#009E73', '#CC79A7', '#56B4E9', '#F0E442']
    rc('axes', prop_cycle=cycler(color=colors_order))

if use_seaborn:
    import seaborn as sns
    
    sns.set_theme(context=seaborn_settings["theme"]["context"] if "context" in seaborn_settings["theme"] else "talk",
                  style=seaborn_settings["theme"]["style"] if "style" in seaborn_settings["theme"] else "whitegrid",
                  palette=seaborn_settings["theme"]["palette"] if "palette" in seaborn_settings["theme"] else None,
                  font=seaborn_settings["theme"]["font"] if "font" in seaborn_settings["theme"] else "Fira Sans",
                  font_scale=seaborn_settings["theme"]["font_scale"] if "font_scale" in seaborn_settings["theme"] else 1.0,
                  color_codes=seaborn_settings["theme"]["color_codes"] if "color_codes" in seaborn_settings["theme"] else True
                  )
    
    sns.set_style(seaborn_settings["theme"]["style"], rc=seaborn_settings["rc_style"])
    sns.set_context(seaborn_settings["theme"]["context"],
                    font_scale=seaborn_settings["theme"]["font_scale"] if "font_scale" in seaborn_settings["theme"] else 1.0)

In [ ]:
source = EventSource(str(input_file_simtel), max_events=max_events)

for event in source:
    pass

In [ ]:
r0_table, r1_table = load_tel_id(file_name = input_file_h5, tel_id = tel_id)

In [ ]:
subarray = SubarrayDescription.from_hdf(input_file_h5)
tel_types = {str(tel): tel.camera.geometry for tel in subarray.telescope_types}.keys()
tel_ids_per_tel_type = {tel_type : subarray.get_tel_ids_for_type(tel_type) for tel_type in tel_types}

## DC to PHE
[back to top](#Table-of-contents)

Only the first telescope for each telescope type is considered.

In [ ]:
for tel_type in tel_types:
    
    plt.figure(figsize=get_fig_size(ratio=1., scale=scale)
    plt.suptitle(tel_type)
    
    tel_id = tel_ids_per_tel_type[tel_type][0] # consider only the first telescope
    dc_to_pe = source.file_.laser_calibrations[tel_id]["calib"]

    for channel, dc_to_pe_per_channel in enumerate(dc_to_pe):
        plt.subplot(1,2,channel+1)
        plt.hist(dc_to_pe_per_channel)
        plt.title(f"channel {channel}")
        plt.xlabel("DC to PHE factor")
        plt.ylabel("Number of pixels")
        plt.grid(visible=True)
        add_stats(dc_to_pe_per_channel, plt.gca(), x=0.62, y=0.90, fontsize=20)
    
    plt.show()

## Pedestals
[back to top](#Table-of-contents)

In [ ]:
for tel_type in tel_types:
    
    plt.figure(figsize=get_fig_size(ratio=1., scale=scale)
    plt.suptitle(tel_type)
    
    tel_id = tel_ids_per_tel_type[tel_type][0] # consider only the first telescope

    pedestals = source.file_.camera_monitorings[tel_id]['pedestal']
    
    for channel, pedestals_per_channel in enumerate(pedestals):
        print(pedestals_per_channel.mean())
        plt.subplot(1,2,channel+1)
        plt.hist(pedestals_per_channel)
        plt.title(f"channel {channel}")
        plt.xlabel("Pedestals")
        plt.ylabel("Number of pixels")
        plt.grid(visible=True)
        add_stats(pedestals_per_channel, plt.gca(), x=0.01, y=0.90, fontsize=15)
    
    plt.show()

## Noise
[back to top](#Table-of-contents)

In [ ]:
for tel_type in tel_types:
    
    tel_id = tel_ids_per_tel_type[tel_type][0] # consider only the first telescope
    noise = source.file_.camera_monitorings[tel_id]['noise']

    if np.all(noise[0] == noise[0][0]) and np.all(noise[1] == noise[1][0]):
        for channel, noise_per_channel in enumerate(noise):
            print(f"Telescope type: {tel_type}, Channel {channel}, Noise: {noise_per_channel[0]} phe")
    else:
        plt.figure(figsize=get_fig_size(ratio=1., scale=scale)
        plt.suptitle(tel_type)
        for channel, noise_per_channel in enumerate(noise):
            plt.subplot(1,2,channel+1)
            plt.hist(noise_per_channel, bins=100)
            plt.title(f"channel {channel}")
            plt.xlabel("Noise")
            plt.ylabel("Number of pixels")
            plt.grid(visible=True)
        plt.show()

## Reference pulse shapes

In [ ]:
for camera_type in subarray.camera_types:
    
    plt.figure(figsize=get_fig_size(ratio=0.5, scale=scale)
    plt.title(camera_type.camera_name)
    
    reference_pulse_shapes = camera_type.readout.reference_pulse_shape
    
    for channel, reference_pulse_shape in enumerate(reference_pulse_shapes):
        
        plt.plot(np.arange(0,len(reference_pulse_shape)), reference_pulse_shape, label=f"Channel {channel}")
        plt.legend()
plt.show()